<a href="https://colab.research.google.com/github/danielahernandez29/Curso-IA/blob/main/RNN_/Inflacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Cargo el archivo
df = pd.read_excel('/content/Consulta_20241113-093525023.xlsx')
df.head(20)

# Uso la fila 8 como encabezado porque alli
new_header = df.iloc[8]
new_header

# Me quedo con los datos a partir de la fila 17 donde empieza la tabla
df = df[17:]
df

# Le asigno los encabezados nuevos al dataframe
df.columns = new_header
df.head()

# Reinicio el indice porque queda todo desordenado después de recortar
df.reset_index(drop=True, inplace=True)
df.head()

# Convertir la columna Título a fecha
df['Fecha'] = pd.to_datetime(df['Título'])
df.head()

df.info()

# Filtro solo los datos entre 2014 y 2024
_df = df[(df['Fecha'] >= '01-01-2014') & (df['Fecha'] <= '12-31-2024')]
_df.head()

_df.tail()

import matplotlib.pyplot as plt

# Obtengo la lista de fechas para graficar
dates = _df["Fecha"].tolist()

# Grafico
plt.figure(figsize=(16, 10))
plt.plot(dates,_df["Índice Nacional de Precios al consumidor, variación anual"], color='blue', label="Variación anual")
plt.title("Índice Nacional de Precios al Consumidor - variación anual")
plt.xlabel("Tiempo")
plt.ylabel("Variación anual")
plt.xticks(rotation=90)
plt.legend()
plt.show()

# Ahora separo variables para entrenar el modelo
X = _df.drop(columns=["Índice Nacional de Precios al consumidor, variación anual", "Título", "Fecha"])
y = _df["Índice Nacional de Precios al consumidor, variación anual"].values

X.shape
y.shape

from sklearn.preprocessing import MinMaxScaler

# aprenda mejor
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.reshape(-1, 1))

import numpy as np

# Ventana de 12 meses o sea un año
window_size = 12
X_rnn = []
y_rnn = []

# Construyo las secuencias temporales para alimentar la RNN
for i in range(window_size, len(X_scaled)):
    X_rnn.append(X_scaled[i - window_size:i])
    y_rnn.append(y_scaled[i])

X_rnn, y_rnn = np.array(X_rnn), np.array(y_rnn)

# Division entre entrenamiento y prueba
split = int(len(X) * 0.8)
X_train, y_train = X_rnn[:split], y_rnn[:split]
X_test, y_test = X_rnn[split:], y_rnn[split:]

X_train.shape
y_test.shape
window_size
X_train.shape[2]

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN, GRU

# Armo mi modelo RNN con varias capas
model = Sequential([
  SimpleRNN(units=60, return_sequences=True, input_shape=(window_size, X_train.shape[2])),
  SimpleRNN(units=30, return_sequences=True),
  SimpleRNN(units=15, return_sequences=False),
  Dense(units=1)
])

from tensorflow.keras.optimizers import Adam

# Compilo el modelo con Adam y MSE o sea regresion
learning_rate = 0.001
adam_optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=adam_optimizer, loss='mean_squared_error')

# Entreno el modelo
model.fit(X_train, y_train, batch_size=1, epochs=10, validation_data=(X_test, y_test))

# Predigo con el modelo
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_original = scaler.inverse_transform(y_test)

# Imprimo para comparar algunas predicciones vs reales
print("Predicciones:", predictions.flatten()[:5])
print("Valores reales:", y_test_original.flatten()[:5])

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, predictions)
print(f"Error Absoluto Medio (MAE): {mae}")

mse = mean_squared_error(y_test, predictions)
print(f"Error Cuadrático Medio (MSE): {mse}")

rmse = np.sqrt(mse)
print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse}")

r2 = r2_score(y_test, predictions)
print(f"Coeficiente de Determinación (R²): {r2}")

# Grafica final comparando lo real contra lo predicho
valid = _df[split:]
valid = valid.reset_index(drop=True)
valid['Predictions'] = np.nan
valid.loc[window_size:, 'Predictions'] = predictions

dates_valid = pd.to_datetime(valid['Fecha']).apply(lambda x: x.strftime('%Y-%m-%d')).tolist()

plt.figure(figsize=(16, 8))
plt.title('Modelo RNN para Predicción de la inflación')
plt.xlabel('Fecha')
plt.ylabel('Inflacion')
plt.plot(dates_valid, valid[["Índice Nacional de Precios al consumidor, variación anual", 'Predictions']])
plt.legend(['Valor Real', 'Predicciones'], loc='lower right')
plt.xticks(rotation=90)
plt.show()
